In [1]:
import re
from utils import apply

In [ ]:
# convert all AA to OH except before R and at the end of a word
# find . -name  '*.yaml' -exec sed -i '' -e "s/AA\([012]\) \([^R]\)/OH\1 \2/g" {} \;

In [11]:
lot = ("OH0", "OH1", "OH2")

In [20]:
overrides = {
    # things that I can't detect automatically
    "BOTSWANA": "B OH0 T S W AA1 N AX",
    
    # things that were just wrong
    'PHOTOCALL': 'F OW1 T OW0 K AO2 L',
}

exclude = """
YACHT
""".split()

def find_palm_vowel(word, pronun, dictionary):
    if word in overrides:
        return overrides[word]
    
    if word in exclude:
        return True

    a_indices = []
    # replace digraphs with single letters so that the index calculation is more reliable
    word_with_better_letters = word.replace("SH", "ʃ").replace("TH", "θ").replace("WH", "ʍ").replace("CH", "ʒ")
    for combo, offset in (("(?<![OEUWʍ])A(?![OIEUW])", 0),):# ("WA(?![UWL])", 1), ("QUA(?![UWL])", 2)):
        a_indices += [m.start() + offset for m in re.finditer(combo, word_with_better_letters)]
            
    if not a_indices:  # no suitable letters found
        return True
    
    present_stresses = []
    for vowel in lot:
        if vowel in pronun:
            stress = vowel[-1]
            present_stresses.append(stress)
    if not present_stresses:
        return True

    leeway = 1 if len(present_stresses) == 1 else 1  # if there is more than one AO, don't get them mixed up

    changed_anything = False
    new_pronun = pronun
    for stress in present_stresses:
        vowel = f"OH{stress}"
        aa_index = new_pronun.split().index(vowel)
        if min(abs(a_index - aa_index) for a_index in a_indices) > leeway:
            print(f"probably no candiate '{word}', letter indices: {a_indices}, sound index: {aa_index}")
            continue
        new_pronun = new_pronun.replace(vowel, f"AA{stress}")
        changed_anything = True
    if not changed_anything:
        return True
    print(f"change '{word.lower().replace('a', 'A')}', new pronun: '{new_pronun.lower().replace('aa', 'AA')}'")
    return True #new_pronun

In [21]:
apply(
    find_palm_vowel,
    save_result=False,
    only_first_file=False,
    mode="transform",
)

change 'AAberg', new pronun: 'AA1 b axr g'
change 'AAchen', new pronun: 'AA1 k ax n'
change 'AAchener', new pronun: 'AA1 k ax n axr'
change 'AAker', new pronun: 'AA1 k axr'
change 'AAmodt', new pronun: 'AA1 m ax t'
change 'AArhus', new pronun: 'AA2 hh uw1 s'
change 'AAse', new pronun: 'AA1 s'
change 'AAsen', new pronun: 'AA1 s ax n'
change 'AbAbA', new pronun: 'ax b AA1 b ax'
change 'AbAbA', new pronun: 'AA1 b ax b ax'
change 'AbAd', new pronun: 'ax b AA1 d'
change 'AbAlos', new pronun: 'AA0 b AA1 l ow0 z'
change 'AbAre', new pronun: 'AA0 b AA1 r iy0'
change 'AbbAdo', new pronun: 'ax b AA1 d ow0'
change 'AbbAs', new pronun: 'ax b AA1 s'
change 'AbbAsi', new pronun: 'AA0 b AA1 s iy0'
change 'AbbAte', new pronun: 'AA1 b ey0 t'
change 'Abbruzzese', new pronun: 'AA0 b r uw0 t s ey1 z iy0'
probably no candiate 'ABDOLLAH', letter indices: [0, 6], sound index: 3
probably no candiate 'ABDOMINAL', letter indices: [0, 7], sound index: 3
change 'AbdullA', new pronun: 'AA0 b d uw1 l ax'
change 'Ab

change 'bAAde', new pronun: 'b AA1 d'
change 'bAAlbek', new pronun: 'b AA1 l b eh0 k'
change 'bAbA', new pronun: 'b AA1 b ax'
change 'bAbAngidA', new pronun: 'b ax b AA1 ng g ih0 d ax'
change 'bAbAr', new pronun: 'b AA2 b AA1 r'
change 'bAbic', new pronun: 'b AA1 b ih0 k'
change 'bAbu', new pronun: 'b AA0 b uw1'
probably no candiate 'BABYLON', letter indices: [1], sound index: 5
change 'bAccArAt', new pronun: 'b AA2 k axr AA1'
change 'bAcci', new pronun: 'b AA1 ch iy0'
change 'bAch', new pronun: 'b AA1 k'
change 'bAchAr', new pronun: 'b AA1 k axr'
change 'bAchelder', new pronun: 'b AA1 k eh0 l d axr'
change 'bAcher', new pronun: 'b AA1 k axr'
change 'bAchmAn', new pronun: 'b AA1 k m ax n'
change 'bAchmAnn', new pronun: 'b AA1 k m ax n'
change 'bAchner', new pronun: 'b AA1 k n axr'
change 'bAchrAch', new pronun: 'b AA1 k r ax k'
probably no candiate 'BACKDROP', letter indices: [1], sound index: 5
probably no candiate 'BACKSTOP', letter indices: [1], sound index: 5
probably no candiate '

change 'cAAn', new pronun: 'k AA1 n'
change 'cAbA', new pronun: 'k AA1 b ax'
change 'cAbAl', new pronun: 'k ax b AA1 l'
change 'cAbelA', new pronun: 'k AA2 b eh1 l ax'
change 'cAbell', new pronun: 'k AA0 b eh1 l'
change 'cAbrAles', new pronun: 'k AA0 b r AA1 l eh0 s'
change 'cAbrerA', new pronun: 'k AA0 b r eh1 r ax'
change 'cAccAmo', new pronun: 'k AA0 k AA1 m ow0'
change 'cAcciA', new pronun: 'k AA1 ch ax'
change 'cAcciAtore', new pronun: 'k AA0 ch ax t ao1 r iy0'
change 'cAcciolA', new pronun: 'k AA0 k ch ow1 l ax'
change 'cAceres', new pronun: 'k AA0 s eh1 r eh0 s'
change 'cAcioppo', new pronun: 'k AA0 ch ow1 p ow0'
probably no candiate 'CACOPHONY', letter indices: [1], sound index: 3
change 'cAdA', new pronun: 'k AA1 d ax'
change 'cAdell', new pronun: 'k AA0 d ey1 l'
change 'cAdogAn', new pronun: 'k AA0 d ow0 g AA1 n'
probably no candiate 'CADOTTE', letter indices: [1], sound index: 3
change 'cAen', new pronun: 'k AA1 n'
change 'cAetAno', new pronun: 'k ax t AA1 n ow0'
change 'cAf

change 'dAAs', new pronun: 'd AA1 s'
change 'dAchA', new pronun: 'd AA1 ch ax'
change 'dAchshund', new pronun: 'd AA1 k s hh uh2 n d'
change 'dAchshunds', new pronun: 'd AA1 k s hh uh2 n t s'
change 'dAcostA', new pronun: 'd AA0 k ow1 s t ax'
probably no candiate 'DACRON', letter indices: [1], sound index: 4
change 'dAdA', new pronun: 'd AA1 d AA2'
change 'dAddArio', new pronun: 'd AA0 d AA1 r iy0 ow0'
change 'dAgostino', new pronun: 'd AA0 g ow0 s t iy1 n ow0'
change 'dAgue', new pronun: 'd AA1 g'
change 'dAher', new pronun: 'd AA1 axr'
change 'dAhl', new pronun: 'd AA1 l'
change 'dAhlberg', new pronun: 'd AA1 l b axr g'
change 'dAhle', new pronun: 'd AA1 ax l'
change 'dAhlem', new pronun: 'd AA1 l ih0 m'
change 'dAhlen', new pronun: 'd AA1 l ax n'
change 'dAhlin', new pronun: 'd AA1 l ih0 n'
change 'dAhlke', new pronun: 'd AA1 l k'
change 'dAhlmAn', new pronun: 'd AA1 l m ax n'
change 'dAhlquist', new pronun: 'd AA1 l k w ih2 s t'
change 'dAhlstrom', new pronun: 'd AA1 l s t r ax m'


probably no candiate 'EARSHOT', letter indices: [1], sound index: 3
probably no candiate 'EAVESDROP', letter indices: [1], sound index: 5
probably no candiate 'EAVESDROPPING', letter indices: [1], sound index: 5
change 'ebAno', new pronun: 'ih0 b AA1 n ow0'
probably no candiate 'ECHOGRAPHY', letter indices: [5], sound index: 2
probably no candiate 'ECOLOGICAL', letter indices: [8], sound index: 4
probably no candiate 'ECOLOGICAL', letter indices: [8], sound index: 4
probably no candiate 'ECOLOGICALLY', letter indices: [8], sound index: 4
probably no candiate 'ECOLOGICALLY', letter indices: [8], sound index: 4
probably no candiate 'ECONOMICAL', letter indices: [8], sound index: 4
probably no candiate 'ECONOMICAL', letter indices: [8], sound index: 4
probably no candiate 'ECONOMICALLY', letter indices: [8], sound index: 4
probably no candiate 'ECONOMICALLY', letter indices: [8], sound index: 4
change 'eisenAch', new pronun: 'ay1 z ax n AA2 k'
change 'eisenbAch', new pronun: 'ay1 z ax n b

...

change 'rAAb', new pronun: 'r AA1 b'
change 'rAAbe', new pronun: 'r AA1 b'
change 'rAAd', new pronun: 'r AA1 d'
change 'rAAsch', new pronun: 'r AA1 sh'
change 'rAbbAni', new pronun: 'r ax b AA1 n iy0'
change 'rAbi', new pronun: 'r AA1 b iy0'
change 'rAbin', new pronun: 'r AA2 b iy1 n'
change 'rAbobAnk', new pronun: 'r AA1 b ow0 b ae2 ng k'
change 'rAbon', new pronun: 'r AA0 b oo1 n'
probably no candiate 'RACICOT', letter indices: [1], sound index: 5
probably no candiate 'RACONTEUR', letter indices: [1], sound index: 3
change 'rAcz', new pronun: 'r AA1 ch'
change 'rAdA', new pronun: 'r AA1 d ax'
change 'rAde', new pronun: 'r AA1 d ey0'
change 'rAdhA', new pronun: 'r AA1 d ax'
change 'rAdi', new pronun: 'r AA1 d iy0'
change 'rAdiogrAphic', new pronun: 'r ey2 d iy0 ow0 g r AA1 f ih0 k'
probably no candiate 'RADIOGRAPHY', letter indices: [1, 7], sound index: 4
probably no candiate 'RADIOLOGICAL', letter indices: [1, 10], sound index: 6
probably no candiate 'RADIOLOGIST', letter indices: [1

change 'sAAb', new pronun: 's AA1 b'
change 'sAAd', new pronun: 's AA1 d'
change 'sAAl', new pronun: 's AA1 l'
change 'sAAm', new pronun: 's AA1 m'
change 'sAAs', new pronun: 's AA1 s'
change 'sAAtchi', new pronun: 's AA1 ch iy0'
change 'sAAthoff', new pronun: 's AA1 t hh oo2 f'
change 'sAAvedrA', new pronun: 's AA0 v ey1 d r ax'
change 'sAbA', new pronun: 's AA1 b ax'
change 'sAbAh', new pronun: 's AA1 b ax'
change 'sAbAt', new pronun: 's AA1 b AA0 t'
change 'sAbAtino', new pronun: 's AA0 b AA0 t iy1 n ow0'
change 'sAbAto', new pronun: 's AA0 b AA1 t ow0'
change 'sAbel', new pronun: 's AA0 b eh1 l'
change 'sAbiA', new pronun: 's AA1 b iy0 ax'
change 'sAbino', new pronun: 's AA0 b iy1 n ow0'
change 'sAbo', new pronun: 's AA1 b ow0'
change 'sAbol', new pronun: 's AA0 b ao1 l'
change 'sAbotAge', new pronun: 's ae1 b ax t AA2 zh'
change 'sAbotAged', new pronun: 's ae1 b ax t AA2 zh d'
change 'sAbotAging', new pronun: 's ae1 b ax t AA2 zh ih0 ng'
change 'sAbrA', new pronun: 's AA1 b r ax'


change 'tAAffe', new pronun: 't AA1 f'
change 'tAbAr', new pronun: 't AA0 b AA1 r'
probably no candiate 'TABLETOP', letter indices: [1], sound index: 6
change 'tAbuchi', new pronun: 't AA2 b uw1 ch iy0'
change 'tAcitA', new pronun: 't AA0 ch iy1 t ax'
change 'tAco', new pronun: 't AA1 k ow0'
change 'tAdA', new pronun: 't AA1 d ax'
change 'tAdAshi', new pronun: 't AA2 d AA1 sh iy0'
change 'tAddeo', new pronun: 't AA1 d iy0 ow0'
probably no candiate 'TAEKWONDO', letter indices: [1], sound index: 4
probably no candiate 'TAEKWONDO', letter indices: [1], sound index: 4
change 'tAfoyA', new pronun: 't AA0 f oy1 ax'
probably no candiate 'TAGALOG', letter indices: [1, 3], sound index: 5
change 'tAgliAferri', new pronun: 't AA0 g l y AA0 f eh1 r iy0'
change 'tAgue', new pronun: 't AA1 g'
change 'tAhir', new pronun: 't AA0 hh ih1 r'
change 'tAirA', new pronun: 't AA0 ih1 r ax'
probably no candiate 'TAIWAN', letter indices: [1], sound index: 3
probably no candiate 'TAIWANESE', letter indices: [1]

change 'ugAlde', new pronun: 'uw0 g AA1 l d iy0'
change 'ugAndA', new pronun: 'y uw2 g AA1 n d ax'
change 'ugAndAn', new pronun: 'y uw2 g AA1 n d ax n'
change 'ulAAnbAAtAr', new pronun: 'uw2 l AA0 n b ax t AA1 r'
probably no candiate 'ULTRAMODERN', letter indices: [4], sound index: 6
probably no candiate 'ULTRASONIC', letter indices: [4], sound index: 6
change 'umAnA', new pronun: 'uw0 m AA1 n ax'
change 'umbAch', new pronun: 'ah1 m b AA2 k'
probably no candiate 'UNAPOLOGETIC', letter indices: [2], sound index: 4
probably no candiate 'UNCOMPENSATED', letter indices: [9], sound index: 3
probably no candiate 'UNCOMPLICATED', letter indices: [9], sound index: 3
probably no candiate 'UNCONSCIONABLE', letter indices: [10], sound index: 3
probably no candiate 'UNCONSOLIDATED', letter indices: [10], sound index: 6
probably no candiate 'UNCONSTITUTIONAL', letter indices: [14], sound index: 3
probably no candiate 'UNCONTROVERSIAL', letter indices: [13], sound index: 3
probably no candiate 'UNCO

change 'wAAg', new pronun: 'w AA1 g'
change 'wAAge', new pronun: 'w AA1 ih0 jh'
change 'wAAl', new pronun: 'w AA1 l'
change 'wAAs', new pronun: 'w AA1 z'
probably no candiate 'WACHOVIA', letter indices: [6], sound index: 1
probably no candiate 'WADA', letter indices: [3], sound index: 1
probably no candiate 'WADDINGHAM', letter indices: [8], sound index: 1
probably no candiate 'WAGA', letter indices: [3], sound index: 1
probably no candiate 'WAGNERIAN', letter indices: [7], sound index: 1
probably no candiate 'WAKABAYASHI', letter indices: [3, 5, 7], sound index: 1
change 'wAkAbAyAshi', new pronun: 'w oh0 k AA2 b oh0 y AA1 sh iy0'
probably no candiate 'WALCZAK', letter indices: [5], sound index: 1
probably no candiate 'WALDEMAR', letter indices: [6], sound index: 1
probably no candiate 'WALDMAN', letter indices: [5], sound index: 1
probably no candiate 'WALLABIES', letter indices: [4], sound index: 1
probably no candiate 'WALLABY', letter indices: [4], sound index: 1
change 'wAmbAch', 

In [17]:
[m.start() for m in re.finditer("(?<![W])A(?![UW])", "WAUSHWA")]

[]